# Loan Payment Data

In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../input/loandata/Loan payments data.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isna().sum()

In [ ]:
df.nunique()

In [ ]:
df[['past_due_days','loan_status']].corr()

In [ ]:
df[df.past_due_days.isnull()]

In [ ]:
df.past_due_days.fillna(0,inplace=True)

In [ ]:
df.nunique()

In [ ]:
df.head()

In [ ]:
df['loan_status'].value_counts().plot(kind='pie',figsize=[7,7],autopct='%.f%%');

In [ ]:
i=1
plt.figure(figsize=[20,15])
if i<=3:
    for col in df[['loan_status','education','Gender']]:
        plt.subplot(4,4,i)
        sns.countplot(x=col,data=df)
        plt.xticks(rotation=45)
        i+=1

In [ ]:
i=1
plt.figure(figsize=[12,15])
if i<=6:
    for col in df[['Principal','terms','past_due_days','age']]:
        plt.subplot(4,2,i)
        sns.histplot(x=col,bins=50,kde=True,color='g',data=df)
        i+=1

In [ ]:
plt.figure(figsize=[12,5])
sns.barplot(x='loan_status',y='age',hue='Gender',data=df);

In [ ]:
df.groupby(['Gender','education'])['loan_status'].count().plot(kind='barh');

In [ ]:
sns.pairplot(df);

In [ ]:
df.drop('effective_date',axis=1, inplace=True)
df.drop('due_date', axis=1, inplace=True)
df.drop('paid_off_time', axis=1, inplace=True)
df.drop('Loan_ID', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr(), annot=True);

## PreProcessing

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
df.Gender = le.fit_transform(df.Gender)
df.terms = le.fit_transform(df.terms)
df.loan_status = df.loan_status.replace(['PAIDOFF', 'COLLECTION', 'COLLECTION_PAIDOFF'],[0,1,2])
df.education = df.education.replace(['High School or Below', 'Bechalor', 'college', 'Master or Above'],[0,2,1,3])
df.past_due_days=df.past_due_days.astype('int64')

In [ ]:
df.info()

In [ ]:
df.head()

## First Modeling

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [ ]:
X = df.drop('loan_status', axis=1)
y = df['loan_status']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [ ]:
models = [LogisticRegression,
          KNeighborsClassifier,
          SVC,
          MLPClassifier,
          DecisionTreeClassifier,
          RandomForestClassifier,
          GradientBoostingClassifier,
          XGBClassifier,
          LGBMClassifier,
          CatBoostClassifier]

In [ ]:
Score_accuracy=[]
Score_f1=[]
ModelName=[]


for model in models:
    alg = model().fit(X_train,y_train)
    y_pred = alg.predict(X_test)
    Score_accuracy.append((accuracy_score(y_test, y_pred))*100)
    Score_f1.append((f1_score(y_test, y_pred, average='weighted'))*100)
    ModelName.append(model.__name__)
    Results = pd.DataFrame({'ModelName':ModelName, 'Score_accuracy':Score_accuracy, 'Score_f1':Score_f1})

In [ ]:
Results

In [ ]:
plt.figure(figsize=[12,8])
plt.subplot(2,2,1)
sns.barplot(x='ModelName',y='Score_accuracy',data=Results.sort_values('Score_accuracy',ascending=False))
plt.title('Score_accuracy')
plt.xticks(rotation=90);
plt.subplot(2,2,2)
sns.barplot(x='ModelName',y='Score_f1',data=Results.sort_values('Score_f1',ascending=False))
plt.title('Score_f1')
plt.xticks(rotation=90);

## Hyperparameter Optimization

In [ ]:
ModelName=[]
Score_f1=[]
Score_accuracy=[]

In [ ]:
# LogisticRegression
tuned=LogisticRegression(solver='liblinear').fit(X_train, y_train)
ModelName.append(LogisticRegression.__name__)
Score_f1.append(f1_score(y_test, tuned.predict(X_test), average='weighted'))
Score_accuracy.append(accuracy_score(y_test, tuned.predict(X_test))) 

In [ ]:
# KNeighborsClassifier
model=KNeighborsClassifier()
params = {'n_neighbors':np.arange(1,25)}
tuned = KNeighborsClassifier(**(GridSearchCV(model,params,cv=5,verbose=2,n_jobs=-1).fit(X_train, y_train).best_params_)).fit(X_train, y_train)
ModelName.append(KNeighborsClassifier.__name__)
Score_f1.append(f1_score(y_test, tuned.predict(X_test), average='weighted'))
Score_accuracy.append(accuracy_score(y_test, tuned.predict(X_test))) 

In [ ]:
# Support Vector Machine
model=SVC()
params = {'C': np.arange(1,10), 'kernel': ['linear', 'rbf']}
tuned = SVC(**(GridSearchCV(model,params,cv=5,verbose=2,n_jobs=-1).fit(X_train, y_train).best_params_)).fit(X_train, y_train)
ModelName.append(SVC.__name__)
Score_f1.append(f1_score(y_test, tuned.predict(X_test), average='weighted'))
Score_accuracy.append(accuracy_score(y_test, tuned.predict(X_test)))

In [ ]:
# MLPClassifier
model=MLPClassifier()
params = {'alpha': [1,0.1,0.01,0.03,0.005,0.0001],'hidden_layer_sizes': [(10,10),(100,100),(3,5)]}
tuned = MLPClassifier(**(GridSearchCV(model,params,cv=5,verbose=2,n_jobs=-1).fit(X_train, y_train).best_params_)).fit(X_train, y_train)
ModelName.append(MLPClassifier.__name__)
Score_f1.append(f1_score(y_test, tuned.predict(X_test), average='weighted'))
Score_accuracy.append(accuracy_score(y_test, tuned.predict(X_test)))

In [ ]:
# DecisionTreeClassifier
model=DecisionTreeClassifier()
params = {'min_samples_split':[2,5,10,20],'max_depth':[3,5,8]}
tuned = DecisionTreeClassifier(**(GridSearchCV(model,params,cv=5,verbose=2,n_jobs=-1).fit(X_train, y_train).best_params_)).fit(X_train, y_train)
ModelName.append(DecisionTreeClassifier.__name__)
Score_f1.append(f1_score(y_test, tuned.predict(X_test), average='weighted'))
Score_accuracy.append(accuracy_score(y_test, tuned.predict(X_test)))

In [ ]:
# RandomForestClassifier
model=RandomForestClassifier()
params = {'n_estimators':[100,200,500],'max_features':[3,5,8],'min_samples_split':[3,8]}
tuned = RandomForestClassifier(**(GridSearchCV(model,params,cv=5,verbose=2,n_jobs=-1).fit(X_train, y_train).best_params_)).fit(X_train, y_train)
ModelName.append(RandomForestClassifier.__name__)
Score_f1.append(f1_score(y_test, tuned.predict(X_test), average='weighted'))
Score_accuracy.append(accuracy_score(y_test, tuned.predict(X_test))) 

In [ ]:
# GradientBoostingClassifier
model=GradientBoostingClassifier()
params = {'n_estimators':[100,300,500],'learning_rate':[0.1,0.01,0.001],'max_depth': [2,3,5]}
tuned = GradientBoostingClassifier(**(GridSearchCV(model,params,cv=5,verbose=2,n_jobs=-1).fit(X_train, y_train).best_params_)).fit(X_train, y_train)
ModelName.append(GradientBoostingClassifier.__name__)
Score_f1.append(f1_score(y_test, tuned.predict(X_test), average='weighted'))
Score_accuracy.append(accuracy_score(y_test, tuned.predict(X_test)))

In [ ]:
# XGBClassifier
tuned=XGBClassifier().fit(X_train, y_train)
ModelName.append(XGBClassifier.__name__)
Score_f1.append(f1_score(y_test, tuned.predict(X_test), average='weighted'))
Score_accuracy.append(accuracy_score(y_test, tuned.predict(X_test))) 

In [ ]:
# LGBMClassifier
model=LGBMClassifier()
params = {'learning_rate':[0.001,0.01,0.1],'n_estimators':[200,500,1000],'max_depth':[1,2,3,5,8]}        
tuned = LGBMClassifier(**(GridSearchCV(model,params,cv=5,verbose=2,n_jobs=-1).fit(X_train, y_train).best_params_)).fit(X_train, y_train)
ModelName.append(LGBMClassifier.__name__)
Score_f1.append(f1_score(y_test, tuned.predict(X_test), average='weighted'))
Score_accuracy.append(accuracy_score(y_test, tuned.predict(X_test)))

In [ ]:
# CatBoostClassifier
model=CatBoostClassifier()
params = {'iterations':[200,500],'learning_rate':[0.01,0.03,],'depth':[4,8]}        
tuned = CatBoostClassifier(**(GridSearchCV(model,params,cv=5,verbose=2,n_jobs=-1).fit(X_train, y_train).best_params_)).fit(X_train, y_train)
ModelName.append(CatBoostClassifier.__name__)
Score_f1.append(f1_score(y_test, tuned.predict(X_test), average='weighted'))
Score_accuracy.append(accuracy_score(y_test, tuned.predict(X_test)))

In [ ]:
Results_best = pd.DataFrame({'ModelName':ModelName, 'Score_f1':Score_f1, 'Score_accuracy':Score_accuracy})
Results_best

In [ ]:
plt.figure(figsize=[15,8])
plt.subplot(2,2,1)
sns.barplot(x='ModelName',y='Score_accuracy',data=Results_best.sort_values('Score_accuracy',ascending=False))
plt.title('Score_accuracy')
plt.xticks(rotation=90);
plt.subplot(2,2,2)
sns.barplot(x='ModelName',y='Score_f1',data=Results_best.sort_values('Score_f1',ascending=False))
plt.title('Score_f1')
plt.xticks(rotation=90);